# Image Classification
This guide demonstrates the seperate full workflow to run MobileNetV1 and PPLCNetV2 models on Arm® Ethos™-U85 processor.  

## PPLCNetV2
### 1. Prepare your models

In [ ]:
# create virtual environment with Python 3.10
!python3.10 -m venv pplcnetv2
!source pplcnetv2/bin/activate
cd pplcnetv2/

In [ ]:
# Install dependencies
!cat <<EOF > pplcnetv2_req.txt
anyio==4.9.0
astor==0.8.1
boto3==1.39.7
botocore==1.39.7
bottle==0.13.4
certifi==2025.7.14
charset-normalizer==3.4.2
common==0.1.2
contourpy==1.3.2
cycler==0.12.1
data==0.4
decorator==5.2.1
dual==0.1.0
dynamo3==0.4.10
exceptiongroup==1.3.0
flywheel==0.5.4
fonttools==4.59.0
funcsigs==1.0.2
h11==0.16.0
httpcore==1.0.9
httpx==0.28.1
idna==3.10
jmespath==1.0.1
kiwisolver==1.4.8
matplotlib==3.10.3
mpmath==1.3.0
numpy==2.2.6
opt-einsum==3.3.0
packaging==25.0
paddlelite==2.13rc0
paddlepaddle==2.6.2
peewee==3.18.2
pillow==11.3.0
pip==25.1
protobuf==6.31.1
prox==0.0.17
pyparsing==3.2.3
PySocks==1.7.1
python-dateutil==2.9.0.post0
python-geoip-python3==1.3
PyYAML==6.0.2
pyzmq==27.0.0
requests==2.32.4
s3transfer==0.13.0
setuptools==78.1.1
six==1.17.0
sniffio==1.3.1
sympy==1.14.0
tight==0.1.0
tqdm==4.67.1
typing_extensions==4.14.1
urllib3==2.5.0
utility==1.0
wheel==0.45.1
x2paddle==1.6.0
zmq==0.0.0
EOF
!pip install -r ./pplcnetv2_req.txt

In [ ]:
# Quantize your model
# Download quantize tool
!git clone https://github.com/PaddlePaddle/PaddleSlim.git
!cd PaddleSlim/
!pip install -e .
# Download PPLCNetV2 model
!cd demo
!mkdir inference_model
!cd inference_model
!wget https://paddle-imagenet-models-name.bj.bcebos.com/dygraph/inference/PPLCNetV2_base_infer.tar
!tar -xf PPLCNetV2_base_infer.tar
!cd ..

In [ ]:
# Download dataset
!mkdir data
!cd data

**Prepare your dataset maunally**

You could choose to download [ILSVRC2012](https://www.image-net.org/challenges/LSVRC/2012/) dataset from [Baidu AI Studio](https://aistudio.baidu.com/datasetdetail/182091) or other avaiable channels ie. Kaggle. 
* Download ILSVRC2012_val_list.txt and ILSVRC2012_val.zip maunally.
* Uname ILSVRC2012_val_list.txt to val_list.txt.
* Unzip ILSVRC2012_val.zip to ILSVRC2012.
* Move val_list.txt into ILSVRC2012 folder.
* Move ILSVRC2012 folder under the created "data" folder above (perhaps under "xxx/pplcnetv2/PaddleSlim/demo/data")

In [ ]:
# Quantize model
!cd ..
!export PYTHONPATH=$PYTHONPATH:$(pwd)
!python3 quant/quant_post/quant_post.py --input_name x --model_path inference_model/PPLCNetV2_base_infer --model_filename inference.pdmodel --save_path PPLCNet --use_gpu FALSE --is_full_quantize TRUE

In [ ]:
# Convert quantized model into .nb format
!paddle_lite_opt --model_dir=./PPLCNet --optimize_out_type=naive_buffer --valid_targets=arm --optimize_out=PPLCNetV2_base_infer_int8_opt # The model file "PPLCNetV2_base_infer_int8_opt.nb" will be generated under current directory.

### 2. Run inference example

In [ ]:
# Download example source code
!git clone https://github.com/Arm-Examples/Paddle-on-Ethos-U.git
!cd Paddle-on-Ethos-U
git lfs pull
# Configure inference environment
!bash install.sh

In [ ]:
# Copy above "PPLCNetV2_base_infer_int8_opt.nb" file into "Paddle-on-Ethos-U/model_zoo/PPLCNetV2_infer_int8"
# Run inference
!bash paddle_verify.sh -m pplcnetv2 -p ./model_zoo/PPLCNetV2_infer_int8/ILSVRC2012_val_00000994.JPEG

## MobileNetV1
### 1. Prepare your models

In [ ]:
# create virtual environment with Python 3.10
!python3.10 -m venv mv1
!source mv1/bin/activate
cd mv1/

In [ ]:
# Install dependencies
!cat <<EOF > mv1_req.txt
anyio==4.9.0
astor==0.8.1
boto3==1.39.7
botocore==1.39.7
bottle==0.13.4
certifi==2025.7.14
charset-normalizer==3.4.2
common==0.1.2
contourpy==1.3.2
cycler==0.12.1
data==0.4
decorator==5.2.1
dual==0.1.0
dynamo3==0.4.10
exceptiongroup==1.3.0
flywheel==0.5.4
fonttools==4.59.0
funcsigs==1.0.2
h11==0.16.0
httpcore==1.0.9
httpx==0.28.1
idna==3.10
jmespath==1.0.1
kiwisolver==1.4.8
matplotlib==3.10.3
mpmath==1.3.0
numpy==2.2.6
opt-einsum==3.3.0
packaging==25.0
paddlelite==2.13rc0
paddlepaddle==2.6.2
peewee==3.18.2
pillow==11.3.0
pip==25.1
protobuf==6.31.1
prox==0.0.17
pyparsing==3.2.3
PySocks==1.7.1
python-dateutil==2.9.0.post0
python-geoip-python3==1.3
PyYAML==6.0.2
pyzmq==27.0.0
requests==2.32.4
s3transfer==0.13.0
setuptools==78.1.1
six==1.17.0
sniffio==1.3.1
sympy==1.14.0
tight==0.1.0
tqdm==4.67.1
typing_extensions==4.14.1
urllib3==2.5.0
utility==1.0
wheel==0.45.1
x2paddle==1.6.0
zmq==0.0.0
EOF
!pip install -r ./mv1_req.txt

In [ ]:
# Quantize your model
# Download quantize tool
!git clone https://github.com/PaddlePaddle/PaddleSlim.git
!cd PaddleSlim/
!pip install -e .
# Download MobileNetV1 model
!cd demo
!mkdir inference_model
!cd inference_model
!wget https://paddle-imagenet-models-name.bj.bcebos.com/dygraph/inference/MobileNetV1_infer.tar
!tar -xf MobileNetV1_infer.tar
!cd ..

In [ ]:
# Download dataset
!mkdir data
!cd data

**Prepare your dataset maunally**

You could choose to download [ILSVRC2012](https://www.image-net.org/challenges/LSVRC/2012/) dataset from [Baidu AI Studio](https://aistudio.baidu.com/datasetdetail/182091) or other avaiable channels ie. Kaggle. 
* Download ILSVRC2012_val_list.txt and ILSVRC2012_val.zip maunally.
* Uname ILSVRC2012_val_list.txt to val_list.txt.
* Unzip ILSVRC2012_val.zip to ILSVRC2012.
* Move val_list.txt into ILSVRC2012 folder.
* Move ILSVRC2012 folder under the created "data" folder above (perhaps under "xxx/mv1/PaddleSlim/demo/data")


In [ ]:
# Quantize model
!cd ..
!export PYTHONPATH=$PYTHONPATH:$(pwd)
!python3 quant/quant_post/quant_post.py --model_path inference_model/MobileNetV1_infer --model_filename inference.pdmodel --save_path MobileNet --use_gpu FALSE

In [ ]:
# Convert quantized model into .nb format
!paddle_lite_opt --model_dir=./MobileNet --optimize_out_type=naive_buffer --valid_targets=arm --optimize_out=mobilenetv1_int8_opt # The model file "mobilenetv1_int8_opt.nb" will be generated under current directory.

### 2. Run inference example

In [ ]:
# Download example source code
!git clone https://github.com/Arm-Examples/Paddle-on-Ethos-U.git
!cd Paddle-on-Ethos-U
# Configure inference environment
!bash install.sh

In [ ]:
# Copy above "mobilenetv1_int8_opt.nb" file into "Paddle-on-Ethos-U/model_zoo/MobileNetV1_infer_int8"
# Run inference
!bash paddle_verify.sh -m mv1 -p ./model_zoo/MobileNetV1_infer_int8/ILSVRC2012_val_00000178.JPEG